# RAG的可控制代理

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/agent_runner/agent_runner_rag_controllable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>

在RAG流程之上添加代理能力可以让您处理更复杂的问题。

但是代理的一个大问题是**缺乏可控性/透明性**。代理可能通过思维链/规划来处理用户查询，这需要反复调用LLM。在这个过程中，很难检查发生了什么，或者在中途停止/纠正执行。

本笔记本向您展示了如何使用我们全新的低级代理API，在RAG流程之上实现可控的逐步执行。

我们将在维基百科文档上展示这一点。


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

## 设置数据

这里我们从维基百科加载一个简单的不同城市的数据集。


In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
# llm = OpenAI(model="gpt-3.5-turbo")llm = OpenAI(model="gpt-4-1106-preview")

```python
# 下载数据
```

在这个部分，我们将学习如何下载数据。


In [ ]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

### 加载数据


In [ ]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

### 构建索引/查询引擎/工具


In [ ]:
import osdef get_tool(name, full_name, documents=None):    if not os.path.exists(f"./data/{name}"):        # 构建向量索引        vector_index = VectorStoreIndex.from_documents(documents)        vector_index.storage_context.persist(persist_dir=f"./data/{name}")    else:        vector_index = load_index_from_storage(            StorageContext.from_defaults(persist_dir=f"./data/{name}"),        )    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=llm)    query_engine_tool = QueryEngineTool(        query_engine=query_engine,        metadata=ToolMetadata(            name=name,            description=(                "提供有关Uber季度财务报表的信息，截至"                f" {full_name}"            ),        ),    )    return query_engine_tool

In [ ]:
march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

In [ ]:
query_engine_tools = [march_tool, june_tool, sept_tool]

## 设置代理

在本节中，我们将定义我们的工具并设置代理。


In [ ]:
from llama_index.core.agent import AgentRunner, ReActAgentfrom llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgentfrom llama_index.agent.openai import OpenAIAgentWorker# openai_step_engine = OpenAIAgentWorker.from_tools(#     query_engine_tools, llm=llm, verbose=True# )# agent = AgentRunner(openai_step_engine)# # alternative# agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)agent_llm = OpenAI(model="gpt-3.5-turbo")# agent_llm = OpenAI(model="gpt-4-1106-preview")agent = ReActAgent.from_tools(    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20)

## 运行一些查询

现在我们来演示一下我们的逐步代理框架的功能。

我们将展示它如何处理复杂的查询，无论是端到端还是逐步进行。

然后我们可以展示如何操纵输出。


### 开箱即用

调用 `chat` 将尝试运行整个任务，并且我们注意到它最终只调用了一个工具。


In [ ]:
response = agent.chat("Analyze the changes in R&D expenditures and revenue")

Thought: To analyze the changes in R&D expenditures and revenue, I can use the quarterly financial data provided by the tools.
Action: march_2022
Action Input: {'input': 'R&D expenditures'}
Observation: Research and development expenses for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. The increase was primarily attributable to a $63 million increase in stock-based compensation expense. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022.
Thought: I have obtained the information about the changes in R&D expenditures for the quarter ending March 2022. Now, I need to gather information about the revenue for the same period to analyze the changes.
Action: march_2022
Action Input: {'input': 'revenue'}
Observation: Revenue for the three months ended March 31, 2022, was $6,854 million, which represents a 136% increase from the $2,903 million reported in the same period i

In [ ]:
print(str(response))

The R&D expenditures for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. This increase was primarily driven by a $63 million increase in stock-based compensation expense. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022.

On the other hand, the revenue for the three months ended March 31, 2022, was $6,854 million, representing a 136% increase from the $2,903 million reported in the same period in 2021. This significant increase in revenue was primarily driven by a 35% increase in Gross Bookings, driven by increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19. Additionally, there was a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace in the fourth quarter of 2021. Furthermore, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business m

### 测试逐步执行

端到端的聊天功能没有起作用。让我们尝试逐步分解，如果出现问题，我们可以注入自己的反馈。


In [ ]:
# 开始任务task = agent.create_task("分析研发支出和收入的变化")

这会返回一个`Task`对象，其中包含`input`、`extra_state`中的附加状态和其他字段。

现在让我们尝试执行这个任务的单个步骤。


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: To analyze the changes in R&D expenditures and revenue, I can use the quarterly financial data provided by the tools.
Action: march_2022
Action Input: {'input': 'R&D expenditures'}
Observation: Research and development expenses for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022. The increase in research and development expenses was primarily due to a $63 million increase in stock-based compensation expense.


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I have obtained the information about the changes in R&D expenditures for the quarter ending March 2022. Now, I need to gather information about the revenue for the same period to analyze the changes in revenue.
Action: march_2022
Action Input: {'input': 'revenue'}
Observation: Revenue for the three months ended March 31, 2022 was $6,854 million, which represents a 136% increase from the $2,903 million reported in the same period in 2021. This increase was primarily attributable to a 35% increase in Gross Bookings, driven by increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19, and a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace. Additionally, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business model changes in the UK.


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I have obtained the information about the changes in R&D expenditures and revenue for the quarter ending March 2022. Now, I can analyze the changes in R&D expenditures and revenue.
Answer: The R&D expenditures for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022. On the other hand, the revenue for the three months ended March 31, 2022, was $6,854 million, representing a 136% increase from the $2,903 million reported in the same period in 2021. This increase in revenue was primarily driven by a 35% increase in Gross Bookings, a $1.5 billion increase in Freight revenue due to the acquisition of Transplace, and increases in Delivery and Mobility revenue due to business model changes in the UK.


我们遇到了**相同的问题**。查询已经完成，尽管我们还没有分析文档！我们可以添加用户输入吗？


In [ ]:
step_output = agent.run_step(task.task_id, input="What about June?")

Added user message to memory: What about June?
Thought: To analyze the changes in R&D expenditures and revenue for June, I need to gather the financial data for that period.
Action: june_2022
Action Input: {'input': 'R&D expenditures'}
Observation: Research and development expenses for the three months ended June 30, 2022, were $704 million, which is a 44% increase from $488 million in the same period in 2021. For the six months ended June 30, 2022, research and development expenses were $1,291 million, up 29% from $1,003 million in the same period in 2021. The increase in expenses for both periods was primarily due to a rise in stock-based compensation and an increase in employee headcount costs.


In [ ]:
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id, input="What about September?")

Added user message to memory: What about September?
Thought: To analyze the changes in R&D expenditures and revenue for September, I need to gather the financial data for that period.
Action: sept_2022
Action Input: {'input': 'R&D expenditures'}
Observation: Research and development expenses for the three months ended September 30, 2022, were $760 million, which is a 54% increase from the $493 million reported for the same period in 2021. For the nine months ended September 30, 2022, research and development expenses were $2,051 million, up 37% from $1,496 million for the same period in 2021. The increase in expenses for both periods was primarily due to a rise in stock-based compensation and an increase in employee headcount costs.


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I have obtained the information about the changes in R&D expenditures and revenue for the quarters ending June and September 2022. Now, I can analyze the changes in R&D expenditures and revenue.
Answer: The R&D expenditures for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022. For the three months ended June 30, 2022, the R&D expenditures increased to $704 million, representing a 44% increase from the same period in 2021. The R&D expenses for the three months ended September 30, 2022, were $760 million, which is a 54% increase from the same period in 2021. 

Regarding revenue, for the three months ended March 31, 2022, the revenue was $6,854 million, representing a 136% increase from the same period in 2021. For the three and six months ended June 30, 2022, the revenue was $8,073 million and $14,927 million,

由于步骤看起来很好，我们现在可以调用`finalize_response`，获取我们的响应。

这也将把任务执行提交到我们`agent_runner`中的`memory`对象中。我们可以检查它。


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

The R&D expenditures for the three months ended March 31 increased from $515 million in 2021 to $587 million in 2022, which is a 14% change. The percentage of revenue allocated to research and development decreased from 18% in 2021 to 9% in 2022. For the three months ended June 30, 2022, the R&D expenditures increased to $704 million, representing a 44% increase from the same period in 2021. The R&D expenses for the three months ended September 30, 2022, were $760 million, which is a 54% increase from the same period in 2021. 

Regarding revenue, for the three months ended March 31, 2022, the revenue was $6,854 million, representing a 136% increase from the same period in 2021. For the three and six months ended June 30, 2022, the revenue was $8,073 million and $14,927 million, respectively, representing increases of 105% and 118% compared to the same periods in 2021. 

The increase in revenue was primarily driven by a 35% increase in Gross Bookings, a $1.5 billion increase in Freight 

## 设置人在循环中的聊天

有了这些功能，可以很容易地在与代理交互时设置人在循环中（或LLM在循环中）的反馈，特别是对于长时间运行的任务。

我们设置了一个双循环：一个用于任务（用户与代理“聊天”），另一个用于控制中间执行。


In [ ]:
agent_llm = OpenAI(model="gpt-3.5-turbo")# agent_llm = OpenAI(model="gpt-4-1106-preview")agent = ReActAgent.from_tools(    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20)

In [ ]:
def chat_repl(exit_when_done: bool = True):    """聊天 REPL。    Args:        exit_when_done(bool): 如果为True，则在步骤完成时自动退出。            如果要继续进行，即使代理已将步骤标记为完成，请将其设置为False。            如果为False，则需要显式调用 "exit" 来完成任务执行。    """    task_message = None    while task_message != "exit":        task_message = input(">> 人类: ")        if task_message == "exit":            break        task = agent.create_task(task_message)        response = None        step_output = None        message = None        while message != "exit":            if message is None or message == "":                step_output = agent.run_step(task.task_id)            else:                step_output = agent.run_step(task.task_id, input=message)            if exit_when_done and step_output.is_last:                print(                    ">> 代理已标记任务为完成，执行任务执行。"                )                break            message = input(                ">> 在步骤中添加反馈？（按回车/留空继续，输入 'exit' 停止）: "            )            if message == "exit":                break        if step_output is None:            print(">> 您尚未运行代理。任务已被丢弃。")        elif not step_output.is_last:            print(">> 代理尚未完成。任务已被丢弃。")        else:            response = agent.finalize_response(task.task_id)        print(f"代理: {str(response)}")

In [ ]:
chat_repl()

>> Human:  What are the risk factors in the last two quarters?


Added user message to memory: What are the risk factors in the last two quarters?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The risk factors affecting the business include:

- Significant reliance on Gross Bookings from trips in large metropolitan areas, which may be negatively affected by various conditions including economic, social, weather, regulatory conditions, and circumstances like COVID-19.
- Potential failure to offer autonomous vehicle technologies on the platform, or offering technologies that may be inferior or perceived as less safe compared to competitors.
- Dependence on retaining and attracting high-quality personnel, with the risk that attrition or unsuccessful succession planning could adversely affect the business.
- Risks of security or data privacy breaches, unauthorized access, alteration, or destruction of proprietary, confidential, employee, or platform user data.
- Thre

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  I meant June and September


Added user message to memory: I meant June and September
Thought: I need to use a tool to help me answer the question.
Action: june_2022
Action Input: {'input': 'risk factors'}
Observation: The risk factors include:

1. Impact of COVID-19 pandemic or future disease outbreaks on business partners and third-party vendors, potentially leading to adverse impacts on the company's business, financial performance, and stock price.
2. Economic conditions affecting discretionary consumer spending, which may lead to shifts in consumer behavior towards lower-cost alternatives or reduced usage of the company's services.
3. Increases in fuel, food, labor, energy, and other costs due to inflation and other factors, which could increase operating costs for drivers, merchants, and carriers, potentially reducing their activity on the platform.
4. Dependence on the performance and reliability of Internet, mobile, and other infrastructures that are not under the company's control.
5. Risks associated wit

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  


Thought: I need to use a tool to help me answer the question.
Action: sept_2022
Action Input: {'input': 'risk factors'}
Observation: Risk factors include:

- Software releases causing interruptions or negative experiences for platform users, potentially leading to loss of users, revenue, and legal or regulatory issues.
- Risks associated with the use of artificial intelligence, such as flawed algorithms, biased datasets, and potential legal and reputational harm due to unintentional bias and discrimination.
- Climate change risks, including physical risks like extreme weather events and transitional risks such as market shifts toward electric vehicles and lower carbon business models.
- Regulatory requirements related to climate change and the company's own climate commitments, which require significant investment and may be impacted by external factors.
- General economic risks, such as the impact of contagious disease outbreaks like the COVID-19 pandemic, which can reduce demand for 

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  


Thought: I can answer without using any more tools.
Response: The risk factors mentioned in both June and September include the impact of the COVID-19 pandemic or future disease outbreaks, economic conditions affecting consumer spending, increases in costs, dependence on external infrastructures, potential engagement in inappropriate activities by platform users, substantial investments in new technologies, concentration of Gross Bookings in large metropolitan areas, risks associated with autonomous vehicle technologies, the need to retain high-quality personnel, security and data privacy breaches, cyberattacks, climate change risks, dependence on third parties, the need for additional capital, risks related to acquisitions and integration, legal and regulatory risks, government regulation and oversight, risks related to data collection and processing, challenges in protecting intellectual property, volatility in stock price, risks in certain jurisdictions, and the impact of catastroph

>> Human:  exit
